In [1]:
import sklearn
from sklearn import datasets
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import word_tokenize        
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import time
from sklearn.feature_extraction import stop_words
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import load_files

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fandi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fandi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fandi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
movie_train=load_files('aclImdb/train/', description=None, 
                                        categories=['neg', 'pos'], load_content=True, shuffle=True,
                                        encoding=None, decode_error='strict', random_state=0)
movie_test=load_files('aclImdb/test/', description=None, 
                                        categories=['neg', 'pos'], load_content=True, shuffle=True,
                                        encoding=None, decode_error='strict', random_state=0)

In [4]:
class LemmaTokenizer:
     def __init__(self):
         self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stoplist,lowercase=True,min_df=3,tokenizer=LemmaTokenizer())),
    ('norm',Normalizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',LogisticRegression()),
 ])

tuned_parameters= [{'clf__C':[1.0,1.5,2.0,2.5,3.0,4.0,5.0],'clf__tol': [1e-4],'clf__max_iter':[400],'clf__solver':['lbfgs']}]
t1_start = time.time() 
clf=GridSearchCV(text_clf, tuned_parameters, scoring='accuracy',cv=5, n_jobs=-1)
clf.fit(movie_train.data,movie_train.target)
print(clf.best_params_)
print(clf.cv_results_['rank_test_score'])
print(clf.cv_results_['mean_test_score'])
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)

In [1]:
lg = LogisticRegression(C= 2.0, max_iter= 40000, solver= 'lbfgs', tol=0.00000000001)
stoplist = nltk.corpus.stopwords.words('english')
count_vect=CountVectorizer(stop_words=stoplist,lowercase=True,tokenizer=LemmaTokenizer())
tfidf_transformer = TfidfTransformer()
#Train data retrieve
X_train_counts = count_vect.fit_transform(movie_train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
lg.fit(X_train_tfidf, movie_train.target)
#Test data retrieve
X_test_counts = count_vect.transform(movie_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

#Print the test accuracy
t1_start = time.time() 
predicted = lg.predict(X_test_tfidf)
t1_end = time.time() 
print(classification_report(movie_test.target, predicted))
print('mean',np.mean(predicted==movie_test.target))
print('processing time is :',t1_end-t1_start)

NameError: name 'LogisticRegression' is not defined

### Decision Tree

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stoplist,lowercase=True)),
    ('norm',Normalizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',tree.DecisionTreeClassifier()),
 ])

tuned_parameters= [{'clf__max_depth':[50,100,200,400,None],'clf__max_features':[None],'clf__splitter':['random','best']}]
t1_start = time.time() 
clf=GridSearchCV(text_clf, tuned_parameters, scoring='accuracy',cv=5, n_jobs=-1)
clf.fit(movie_train.data,movie_train.target)
print(clf.best_params_)
print(clf.cv_results_['rank_test_score'])
print(clf.cv_results_['mean_test_score'])
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)

In [ ]:
#applied sub and subtest as the traindata and testdata, improved accuracy by 0.005
count_vect=CountVectorizer(stop_words=stoplist,lowercase=True)
tfidf_transformer = TfidfTransformer()
X_train_counts = count_vect.fit_transform(movie_train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#Y_train_tfidf = tfidf_transformer.fit_transform(Y_train_counts)

X_test_counts = count_vect.transform(movie_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

DT_clf = tree.DecisionTreeClassifier(max_depth=None, max_features=None, max_leaf_nodes=None, splitter='random')
DT_clf=DT_clf.fit(X_train_tfidf,movie_train.target )
predicted = DT_clf.predict(X_test_tfidf)
print(classification_report(movie_test.target, predicted))
print(np.mean(predicted ==movie_test.target))

### Adaboost

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')

text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stoplist,lowercase=True)),
    ('norm',Normalizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',AdaBoostClassifier()),
 ])

tuned_parameters= [{'clf__learning_rate':[0.1,0.2,0.5,1.0],'clf__n_estimators':[50,100]}]
t1_start = time.time() 
clf=GridSearchCV(text_clf, tuned_parameters, scoring='accuracy',cv=5, n_jobs=-1)
clf.fit(movie_train.data,movie_train.target)
print(clf.best_params_)
print(clf.cv_results_['rank_test_score'])
print(clf.cv_results_['mean_test_score'])
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')
count_vect=CountVectorizer(stop_words=stoplist,lowercase=True)
tfidf_transformer = TfidfTransformer()
X_train_counts = count_vect.fit_transform(movie_train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#Y_train_tfidf = tfidf_transformer.fit_transform(Y_train_counts)

X_test_counts = count_vect.transform(movie_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
t1_start = time.time() 
DT_clf = AdaBoostClassifier(learning_rate=0.5,n_estimators=1000)
DT_clf=DT_clf.fit(X_train_tfidf,movie_train.target )
predicted = DT_clf.predict(X_test_tfidf)
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)
print(classification_report(movie_test.target, predicted))
print(np.mean(predicted ==movie_test.target))

### Linear SVC

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')

text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stoplist,lowercase=True)),
    ('norm',Normalizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',LinearSVC()),
 ])

tuned_parameters= [{'clf__loss':['hinge','squared_hinge'],'clf__tol':[1e-4,1e-6,1e-7,1e-9,1e-11],'clf__max_iter':[100000]}]
t1_start = time.time() 
clf=GridSearchCV(text_clf, tuned_parameters, scoring='accuracy',cv=5, n_jobs=-1)
clf.fit(movie_train.data,movie_train.target)
print(clf.best_params_)
print(clf.cv_results_['rank_test_score'])
print(clf.cv_results_['mean_test_score'])
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')
count_vect=CountVectorizer(stop_words=stoplist,lowercase=True)
tfidf_transformer = TfidfTransformer()
X_train_counts = count_vect.fit_transform(movie_train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#Y_train_tfidf = tfidf_transformer.fit_transform(Y_train_counts)

X_test_counts = count_vect.transform(movie_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
t1_start = time.time() 
SVC_clf = LinearSVC(loss='squared_hinge',max_iter=100000,tol= 0.0001)
SVC_clf=SVC_clf.fit(X_train_tfidf,movie_train.target )
predicted = SVC_clf.predict(X_test_tfidf)
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)
print(classification_report(movie_test.target, predicted))
print(np.mean(predicted ==movie_test.target))

### Random Forest

In [3]:
stoplist = nltk.corpus.stopwords.words('english')

text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stoplist,lowercase=True)),
    ('norm',Normalizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',RandomForestClassifier()),
 ])

tuned_parameters= [{'clf__n_estimators':[50,100,200,300],'clf__max_depth':[1,None]}]
t1_start = time.time() 
clf=GridSearchCV(text_clf, tuned_parameters, scoring='accuracy',cv=5, n_jobs=-1)
clf.fit(movie_train.data,movie_train.target)
print(clf.best_params_)
print(clf.cv_results_['rank_test_score'])
print(clf.cv_results_['mean_test_score'])
t1_end = time.time() 

KeyboardInterrupt: 

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')
count_vect=CountVectorizer(stop_words=stoplist,lowercase=True)
tfidf_transformer = TfidfTransformer()
X_train_counts = count_vect.fit_transform(movie_train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#Y_train_tfidf = tfidf_transformer.fit_transform(Y_train_counts)

X_test_counts = count_vect.transform(movie_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
t1_start = time.time() 
RF_clf = RandomForestClassifier(max_depth=None,n_estimators= 300,n_jobs=-1)
RF_clf=RF_clf.fit(X_train_tfidf,movie_train.target )
predicted = RF_clf.predict(X_test_tfidf)
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)
print(classification_report(movie_test.target, predicted))
print(np.mean(predicted ==movie_test.target))

### Multinomial NB

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')
stoplist.append('\n')
stoplist.append(' ')
stoplist.append('"')
stoplist.append('|')
stoplist.append('/')
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stoplist,lowercase=True)),
    ('norm',Normalizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',MultinomialNB()),
 ])

tuned_parameters= [{'clf__alpha':[0.01,0.015,0.0195,0.02,0.025,0.03,0.04,0.05]}]
t1_start = time.time() 
clf=GridSearchCV(text_clf, tuned_parameters, scoring='accuracy',cv=5, n_jobs=-1)
clf.fit(movie_train.data,movie_train.target)
print(clf.best_params_)
print(clf.cv_results_['rank_test_score'])
print(clf.cv_results_['mean_test_score'])
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)

In [ ]:
stoplist = nltk.corpus.stopwords.words('english')

count_vect=CountVectorizer(stop_words=stoplist,lowercase=True)
tfidf_transformer = TfidfTransformer()
X_train_counts = count_vect.fit_transform(train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#Y_train_tfidf = tfidf_transformer.fit_transform(Y_train_counts)

X_test_counts = count_vect.transform(movie_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
t1_start = time.time() 
RF_clf = MultinomialNB(alpha=0.0190)
RF_clf=RF_clf.fit(X_train_tfidf,movie_train.target )
predicted = RF_clf.predict(X_test_tfidf)
t1_end = time.time() 
print('processing time is :',t1_end-t1_start)
print(classification_report(movie_test.target, predicted))
print(np.mean(predicted ==movie_test.target))